# zip_is_different_in_TF_and_python

* The Q source

https://stackoverflow.com/questions/75244920/split-my-dataset-in-train-validation-using-mapdataset-in-python


Hi everyone I'm facing an issue after that I elaborate images and labels. To create an unique dataset I use the zip function. After the elaboration both images and labels are 18k and it's correct but when I call the zip(image,labels), items become 563. Here some code to let you to understand:

```Python
# Map the load_and_preprocess_image function over the dataset of image paths
images = image_paths.map(load_and_preprocess_image)
# Map the extract_label function over the dataset of image paths
labels = image_paths.map(extract_label)    
# Zip the labels and images together to create a dataset of (image, label) pairs
#HERE SOMETHING STRANGE HAPPENS
data = tf.data.Dataset.zip((images,labels))
# Shuffle and batch the data
data = data.shuffle(buffer_size=1000).batch(32)
# Split the data into train and test sets
data = data.shuffle(buffer_size=len(data))
# Convert the dataset into a collection of data
num_train = int(0.8 * len(data))
train_data = image_paths.take(num_train)
val_data = image_paths.skip(num_train)
```

I cannot see where is the error. Can you help me plese? Thanks

I'd like to have a dataset of 18k images,labels

## my ans

`tf.data.Dataset.zip` is not like Python's `zip`. The `tf.data.Dataset.zip`'s input is `tf datasets`, not the elements of the `tf dataset`. You may check the [official doc][1] for detail. 

In your case, combine the image and label processing in one map function and return both:

    # load_and_preprocess_image_and_label
    def load_and_preprocess_image_and_label(image_path):
        """ load image and label then some operations """
        return image, label
    
    # Map the load_and_preprocess_image function over the dataset of image/label paths
    train_list = tf.data.Dataset.list_files(str(PATH / 'train/*.jpg'))
    data = train_list.map(load_and_preprocess_image_and_label,
                                      num_parallel_calls=tf.data.AUTOTUNE)


  [1]: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#zip

## refernce

https://blog.csdn.net/xinjieyuan/article/details/103749167

```dataset.zip()与zip()```
这个函数和python中的zip()相当的不一样，切勿被迷惑了双眼。首先tensorflow中的zip接受的数据是dataset
作用：通过将给定的数据集压缩在一起创建一个“数据集”。
使用过程举例

In [1]:
import tensorflow as tf
import numpy as np

2023-01-26 20:52:06.445946: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 1. tf.data.dataset input

In [66]:
a = tf.data.Dataset.range(1,4) # [ 1, 2, 3 ]
b = tf.data.Dataset.range(4,7) # [ 4, 5, 6 ]

In [67]:
print(a)

<RangeDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


In [68]:
print("tf.dataset a=", list(a.as_numpy_iterator()))
print("tf.dataset b=", list(b.as_numpy_iterator()))

tf.dataset a= [1, 2, 3]
tf.dataset b= [4, 5, 6]


In [74]:
c = tf.data.Dataset.zip((a, b))
print("c=", list(c.as_numpy_iterator()))

TypeError: Invalid `datasets`. `datasets` is expected to be a (nested) structure of `tf.data.Dataset` objects. Python `list` is not supported and you should use `tuple` instead.

### 2. python range input

In [72]:
# a = range(1,4) # [ 1, 2, 3 ]
# b = range(4,7) # [ 4, 5, 6 ]

a = [ 1, 2, 3 ]
b = [ 4, 5, 6 ]

In [73]:
# for i in a:
#     print(i)

print("a=", list(a))
print("b=", list(b))

a= [1, 2, 3]
b= [4, 5, 6]


In [61]:
c = zip((a, b))

In [62]:
for x in c:
    print(x)

([1, 2, 3],)
([4, 5, 6],)


#### 這是？？

In [65]:
c = zip(a, b)

In [64]:
for x in c:
    print(x)

(1, 4)
(2, 5)
(3, 6)


In [52]:
# c = tf.data.Dataset.zip((a, b))
# print("c=", list(c.as_numpy_iterator()))

## tf.data.Dataset.zip from TF offical doc
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#zip

In [18]:
# The nested structure of the `datasets` argument determines the
# structure of elements in the resulting dataset.
a = tf.data.Dataset.range(1, 4)  # ==> [ 1, 2, 3 ]
b = tf.data.Dataset.range(4, 7)  # ==> [ 4, 5, 6 ]
ds = tf.data.Dataset.zip((a, b))

print("a=", list(a.as_numpy_iterator()))
print("b=", list(b.as_numpy_iterator()))
print("ds=", list(ds.as_numpy_iterator()))

a= [1, 2, 3]
b= [4, 5, 6]
ds= [(1, 4), (2, 5), (3, 6)]


In [19]:
ds = tf.data.Dataset.zip((b, a))
list(ds.as_numpy_iterator())

[(4, 1), (5, 2), (6, 3)]

In [20]:
# The `datasets` argument may contain an arbitrary number of datasets.
c = tf.data.Dataset.range(7, 13).batch(2)  # ==> [ [7, 8],
                                           #       [9, 10],
                                           #       [11, 12] ]
ds = tf.data.Dataset.zip((a, b, c))
for element in ds.as_numpy_iterator():
  print(element)

(1, 4, array([7, 8]))
(2, 5, array([ 9, 10]))
(3, 6, array([11, 12]))


In [21]:
# The number of elements in the resulting dataset is the same as
# the size of the smallest dataset in `datasets`.
d = tf.data.Dataset.range(13, 15)  # ==> [ 13, 14 ]
ds = tf.data.Dataset.zip((a, d))
list(ds.as_numpy_iterator())


[(1, 13), (2, 14)]